# Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá

Las tres tareas se completaron en el mismo notebook. Se incluyen anotaciones para marcar el inicio y fin de cada tarea.

## Paso 1: Importar y procesar la tabla de Wikipedia

In [1]:
#Importar las librerías necesarias
import numpy as np # librería para manejar datos vectorizados
import pandas as pd # librería para análisis de datos
import folium
from geopy.geocoders import Nominatim
import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas
from sklearn.cluster import KMeans
import matplotlib.cm as cm # Matplotlib y módulos asociados para graficar
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#Leer todas las tablas de la página en un DataFrame. Se genera una lista de DataFrames
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
#Ver cuantos Dataframes se generaron
len(df)

3

In [4]:
#Ver el contenido del primer DataFrame
df[0]

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned
5,M1HScarborough(Cedarbrae),M2HNorth York(Hillcrest Village),M3HNorth York(Bathurst Manor / Wilson Heights ...,M4HEast York(Thorncliffe Park),M5HDowntown Toronto(Richmond / Adelaide / King),M6HWest Toronto(Dufferin / Dovercourt Village),M7HNot assigned,M8HNot assigned,M9HNot assigned
6,M1JScarborough(Scarborough Village),M2JNorth York(Fairview / Henry Farm / Oriole),M3JNorth York(Northwood Park / York University),M4JEast YorkEast Toronto(The Danforth East),M5JDowntown Toronto(Harbourfront East / Union ...,M6JWest Toronto(Little Portugal / Trinity),M7JNot assigned,M8JNot assigned,M9JNot assigned
7,M1KScarborough(Kennedy Park / Ionview / East B...,M2KNorth York(Bayview Village),M3KNorth York(Downsview)East (CFB Toronto),M4KEast Toronto(The Danforth West / Riverdale),M5KDowntown Toronto(Toronto Dominion Centre / ...,M6KWest Toronto(Brockton / Parkdale Village / ...,M7KNot assigned,M8KNot assigned,M9KNot assigned
8,M1LScarborough(Golden Mile / Clairlea / Oakridge),M2LNorth York(York Mills / Silver Hills),M3LNorth York(Downsview)West,M4LEast Toronto(India Bazaar / The Beaches West),M5LDowntown Toronto(Commerce Court / Victoria ...,M6LNorth York(North Park / Maple Leaf Park / U...,M7LNot assigned,M8LNot assigned,M9LNorth York(Humber Summit)
9,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M2MNorth York(Willowdale / Newtonbrook),M3MNorth York(Downsview)Central,M4MEast Toronto(Studio District),M5MNorth York(Bedford Park / Lawrence Manor East),M6MYork(Del Ray / Mount Dennis / Keelsdale and...,M7MNot assigned,M8MNot assigned,M9MNorth York(Humberlea / Emery)


Se observa que en el primer DataFrame están los datos buscados, en una tabla de 19x9. Comenzaremos a trabajar con ese DataFrame

In [5]:
df_canada_crudo = df[0]

In [6]:
df_canada_crudo.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


Debemos pasar de una tabla de 8 columnas a otra con una sola columna. Para eso transformamos cada columna en una lista, concatenamos todas las listas y creamos un nuevo DataFrame de una sola columna.

In [7]:
lista_columnas = df_canada_crudo.columns.tolist()
lista_columnas

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [8]:
lista_df = []

In [9]:
for c in lista_columnas:
    lista_df = lista_df + df_canada_crudo[c].tolist()
lista_df

['M1ANot assigned',
 'M1BScarborough(Malvern / Rouge)',
 'M1CScarborough(Rouge Hill / Port Union / Highland Creek)',
 'M1EScarborough(Guildwood / Morningside / West Hill)',
 'M1GScarborough(Woburn)',
 'M1HScarborough(Cedarbrae)',
 'M1JScarborough(Scarborough Village)',
 'M1KScarborough(Kennedy Park / Ionview / East Birchmount Park)',
 'M1LScarborough(Golden Mile / Clairlea / Oakridge)',
 'M1MScarborough(Cliffside / Cliffcrest / Scarborough Village West)',
 'M1NScarborough(Birch Cliff / Cliffside West)',
 'M1PScarborough(Dorset Park / Wexford Heights / Scarborough Town Centre)',
 'M1RScarborough(Wexford / Maryvale)',
 'M1SScarborough(Agincourt)',
 "M1TScarborough(Clarks Corners / Tam O'Shanter / Sullivan)",
 "M1VScarborough(Milliken / Agincourt North / Steeles East / L'Amoreaux East)",
 "M1WScarborough(Steeles West / L'Amoreaux West)",
 'M1XScarborough(Upper Rouge)',
 'M1YNot assigned',
 'M1ZNot assigned',
 'M2ANot assigned',
 'M2BNot assigned',
 'M2CNot assigned',
 'M2ENot assigned',
 

In [10]:
df_canada_pc = pd.DataFrame(lista_df,columns=['crudo'])
df_canada_pc

,crudo
0,M1ANot assigned
1,M1BScarborough(Malvern / Rouge)
2,M1CScarborough(Rouge Hill / Port Union / Highl...
3,M1EScarborough(Guildwood / Morningside / West ...
4,M1GScarborough(Woburn)
5,M1HScarborough(Cedarbrae)
6,M1JScarborough(Scarborough Village)
7,M1KScarborough(Kennedy Park / Ionview / East B...
8,M1LScarborough(Golden Mile / Clairlea / Oakridge)
9,M1MScarborough(Cliffside / Cliffcrest / Scarbo...


Ahora comenzamos a separar esa única columna en todas las columnas que necesitamos. Primero extraemos el codigo postal, que son los primeros tres caracteres.

In [11]:
df_canada_pc['PostalCode'] = df_canada_pc['crudo'].apply(lambda x: x[0:3])
df_canada_pc

,crudo,PostalCode
0,M1ANot assigned,M1A
1,M1BScarborough(Malvern / Rouge),M1B
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C
3,M1EScarborough(Guildwood / Morningside / West ...,M1E
4,M1GScarborough(Woburn),M1G
5,M1HScarborough(Cedarbrae),M1H
6,M1JScarborough(Scarborough Village),M1J
7,M1KScarborough(Kennedy Park / Ionview / East B...,M1K
8,M1LScarborough(Golden Mile / Clairlea / Oakridge),M1L
9,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M1M


Ponemos todos los demás caracteres en otra columna, que contiene municipio y barrio

In [12]:
df_canada_pc['barrio_crudo'] = df_canada_pc['crudo'].apply(lambda x: x[3:])
df_canada_pc

,crudo,PostalCode,barrio_crudo
0,M1ANot assigned,M1A,Not assigned
1,M1BScarborough(Malvern / Rouge),M1B,Scarborough(Malvern / Rouge)
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C,Scarborough(Rouge Hill / Port Union / Highland...
3,M1EScarborough(Guildwood / Morningside / West ...,M1E,Scarborough(Guildwood / Morningside / West Hill)
4,M1GScarborough(Woburn),M1G,Scarborough(Woburn)
5,M1HScarborough(Cedarbrae),M1H,Scarborough(Cedarbrae)
6,M1JScarborough(Scarborough Village),M1J,Scarborough(Scarborough Village)
7,M1KScarborough(Kennedy Park / Ionview / East B...,M1K,Scarborough(Kennedy Park / Ionview / East Birc...
8,M1LScarborough(Golden Mile / Clairlea / Oakridge),M1L,Scarborough(Golden Mile / Clairlea / Oakridge)
9,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M1M,Scarborough(Cliffside / Cliffcrest / Scarborou...


Eliminamos todos los valores **Not assigned**

In [13]:
df_canada_pc = df_canada_pc[df_canada_pc['barrio_crudo'] != 'Not assigned']
df_canada_pc

,crudo,PostalCode,barrio_crudo
1,M1BScarborough(Malvern / Rouge),M1B,Scarborough(Malvern / Rouge)
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C,Scarborough(Rouge Hill / Port Union / Highland...
3,M1EScarborough(Guildwood / Morningside / West ...,M1E,Scarborough(Guildwood / Morningside / West Hill)
4,M1GScarborough(Woburn),M1G,Scarborough(Woburn)
5,M1HScarborough(Cedarbrae),M1H,Scarborough(Cedarbrae)
6,M1JScarborough(Scarborough Village),M1J,Scarborough(Scarborough Village)
7,M1KScarborough(Kennedy Park / Ionview / East B...,M1K,Scarborough(Kennedy Park / Ionview / East Birc...
8,M1LScarborough(Golden Mile / Clairlea / Oakridge),M1L,Scarborough(Golden Mile / Clairlea / Oakridge)
9,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M1M,Scarborough(Cliffside / Cliffcrest / Scarborou...
10,M1NScarborough(Birch Cliff / Cliffside West),M1N,Scarborough(Birch Cliff / Cliffside West)


Hacemos un split del campo con el caracter **(**. Todo lo que queda a la izquierda es el municipio, lo que queda a la derecha es el barrio. Guardamos los datos en listas.

In [14]:
lista_barrios = df_canada_pc.barrio_crudo.str.split('(').tolist()
boroughs = []
neighborhoods = []
for barrio in lista_barrios:
    boroughs.append(barrio[0])
    neighborhoods.append(barrio[1])

In [15]:
boroughs

['Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'East York',
 'East York',
 'East Toronto',
 'East York',
 'East York',
 'East YorkEast Toronto',
 'East Toronto',
 'East Toronto',
 'East Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',

In [16]:
neighborhoods

['Malvern / Rouge)',
 'Rouge Hill / Port Union / Highland Creek)',
 'Guildwood / Morningside / West Hill)',
 'Woburn)',
 'Cedarbrae)',
 'Scarborough Village)',
 'Kennedy Park / Ionview / East Birchmount Park)',
 'Golden Mile / Clairlea / Oakridge)',
 'Cliffside / Cliffcrest / Scarborough Village West)',
 'Birch Cliff / Cliffside West)',
 'Dorset Park / Wexford Heights / Scarborough Town Centre)',
 'Wexford / Maryvale)',
 'Agincourt)',
 "Clarks Corners / Tam O'Shanter / Sullivan)",
 "Milliken / Agincourt North / Steeles East / L'Amoreaux East)",
 "Steeles West / L'Amoreaux West)",
 'Upper Rouge)',
 'Hillcrest Village)',
 'Fairview / Henry Farm / Oriole)',
 'Bayview Village)',
 'York Mills / Silver Hills)',
 'Willowdale / Newtonbrook)',
 'Willowdale)South',
 'York Mills West)',
 'Willowdale)West',
 'Parkwoods)',
 'Don Mills)North',
 'Don Mills)South',
 'Bathurst Manor / Wilson Heights / Downsview North)',
 'Northwood Park / York University)',
 'Downsview)East ',
 'Downsview)West',
 'Down

Eliminamos el caracter **)** del barrio, y todo lo que esté después del mismo. Reemplazamos **/** por **,**

In [17]:
neighborhoods = [n.split(')')[0] for n in neighborhoods]
neighborhoods

['Malvern / Rouge',
 'Rouge Hill / Port Union / Highland Creek',
 'Guildwood / Morningside / West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'Kennedy Park / Ionview / East Birchmount Park',
 'Golden Mile / Clairlea / Oakridge',
 'Cliffside / Cliffcrest / Scarborough Village West',
 'Birch Cliff / Cliffside West',
 'Dorset Park / Wexford Heights / Scarborough Town Centre',
 'Wexford / Maryvale',
 'Agincourt',
 "Clarks Corners / Tam O'Shanter / Sullivan",
 "Milliken / Agincourt North / Steeles East / L'Amoreaux East",
 "Steeles West / L'Amoreaux West",
 'Upper Rouge',
 'Hillcrest Village',
 'Fairview / Henry Farm / Oriole',
 'Bayview Village',
 'York Mills / Silver Hills',
 'Willowdale / Newtonbrook',
 'Willowdale',
 'York Mills West',
 'Willowdale',
 'Parkwoods',
 'Don Mills',
 'Don Mills',
 'Bathurst Manor / Wilson Heights / Downsview North',
 'Northwood Park / York University',
 'Downsview',
 'Downsview',
 'Downsview',
 'Downsview',
 'Victoria Village',
 'Parkview Hill /

In [18]:
neighborhoods = [n.replace('/',',') for n in neighborhoods]
neighborhoods

['Malvern , Rouge',
 'Rouge Hill , Port Union , Highland Creek',
 'Guildwood , Morningside , West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'Kennedy Park , Ionview , East Birchmount Park',
 'Golden Mile , Clairlea , Oakridge',
 'Cliffside , Cliffcrest , Scarborough Village West',
 'Birch Cliff , Cliffside West',
 'Dorset Park , Wexford Heights , Scarborough Town Centre',
 'Wexford , Maryvale',
 'Agincourt',
 "Clarks Corners , Tam O'Shanter , Sullivan",
 "Milliken , Agincourt North , Steeles East , L'Amoreaux East",
 "Steeles West , L'Amoreaux West",
 'Upper Rouge',
 'Hillcrest Village',
 'Fairview , Henry Farm , Oriole',
 'Bayview Village',
 'York Mills , Silver Hills',
 'Willowdale , Newtonbrook',
 'Willowdale',
 'York Mills West',
 'Willowdale',
 'Parkwoods',
 'Don Mills',
 'Don Mills',
 'Bathurst Manor , Wilson Heights , Downsview North',
 'Northwood Park , York University',
 'Downsview',
 'Downsview',
 'Downsview',
 'Downsview',
 'Victoria Village',
 'Parkview Hill ,

Finalmente agregamos las listas al Dataframe y borramos las columnas sobrantes. Con eso queda el DataFrame completo.

In [19]:
df_canada_pc['Borough'] = boroughs

C:\Users\u199401\AppData\Local\Temp/ipykernel_16284/1312262081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_canada_pc['Borough'] = boroughs


In [20]:
df_canada_pc['Neighborhood'] = neighborhoods

C:\Users\u199401\AppData\Local\Temp/ipykernel_16284/77073350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_canada_pc['Neighborhood'] = neighborhoods


In [21]:
df_canada_pc.drop(columns=['crudo','barrio_crudo'],inplace=True)

C:\Users\u199401\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### En la siguiente celda se muestra la respuesta a la parte 1 de la tarea

In [22]:
df_canada_pc

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,"Malvern , Rouge"
2,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
3,M1E,Scarborough,"Guildwood , Morningside , West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
8,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
9,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
10,M1N,Scarborough,"Birch Cliff , Cliffside West"


## Paso 2: Importar las coordenadas de los barrios

La librería GeoCoder siempre falla con un error **<[REQUEST_DENIED] Google - Geocode [empty]>**, por lo que usaremos el CSV con los datos de ubicación

In [23]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Para combinar ambos DataFrames usamos la función merge

In [24]:
df_canada_pc = df_canada_pc.merge(df_coords,left_on='PostalCode',right_on='Postal Code')

In [25]:
df_canada_pc.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


Eliminamos la columna redundante

In [26]:
df_canada_pc.drop(columns=['Postal Code'],inplace=True)

### En la siguiente celda se muestra la respuesta a la parte 2 de la tarea

In [27]:
df_canada_pc

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


## Paso 3: Agrupar los vecindarios de Toronto

Primero nos vamos a quedar solo con los municipio que contienen la palabra Toronto

In [28]:
df_toronto_pc = df_canada_pc[df_canada_pc['Borough'].str.contains('Toronto')].reset_index()

In [29]:
df_toronto_pc

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,40,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
2,41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
3,42,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
4,43,M4M,East Toronto,Studio District,43.659526,-79.340923
5,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
6,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
7,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
8,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
9,48,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160


Mostramos todos los vecindarios en un mapa

In [31]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6541457, -79.38079276646786.


In [32]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(df_toronto_pc['Latitude'], df_toronto_pc['Longitude'], df_toronto_pc['Borough'], df_toronto_pc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Para traer info de lugares en cada barrio, debemos usar la v3 de FourSquare, ya que la v2 está en estado Deprecated y no se puede usar para proyectos nuevos. Se adapataron los scripts para utilizar la nueva version de la API.

In [33]:
headers = {
    "Accept": "application/json",
    "Authorization": "fsq3KGiYJYg1iegiiwPXIQZje9pe2812afytEt727CI4OAU=" #El nuevo metodo de autenticacion es con un token
}
LIMIT = 50 #La nueva API solo acepta LIMIT menor o igual que 50

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API v3, los parametros se pasan en un diccionario y el token en el header
        url = "https://api.foursquare.com/v3/places/search"
        coord = '{},{}'.format(lat,lng)
        params = {"ll": coord,
          "limit": LIMIT,
          "radius": radius}
            
        # solicitud GET
        results = requests.request("GET", url, params=params, headers=headers).json()
        venues = results['results']

        
        # regresa solo información relevante de cada sitio cercano, cuando hay un sitio sin categoría da error y lo ignoramos
        try:
            for v in venues:
                venues_list.append([(
                    name,
                    lat,
                    lng,
                    v['name'],
                    v['geocodes']['main']['latitude'],
                    v['geocodes']['main']['longitude'],
                    v['categories'][0]['name'])])
        except:
            pass
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
toronto_venues = getNearbyVenues(names=df_toronto_pc['Neighborhood'],
                                   latitudes=df_toronto_pc['Latitude'],
                                   longitudes=df_toronto_pc['Longitude']
                                  )

The Beaches
The Danforth East
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Enclave of M5E
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition 

In [36]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676676,-79.294126,Hiking Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.674403,-79.294516,Park
2,The Beaches,43.676357,-79.293031,Santon Audio,43.677003,-79.294270,Audiovisual Service
3,The Beaches,43.676357,-79.293031,Loach Engineering Inc,43.675253,-79.293621,Sports and Recreation
4,The Beaches,43.676357,-79.293031,Verite Films Inc,43.675897,-79.295593,Film Studio
5,The Beaches,43.676357,-79.293031,The Big Carrot Beach Community Market,43.678841,-79.297600,Grocery Store / Supermarket
6,The Danforth East,43.685347,-79.338106,Gabe's Playground,43.686093,-79.337289,Playground
7,The Danforth East,43.685347,-79.338106,Treehouse Collectibles,43.685971,-79.339749,Toy / Game Store
8,The Danforth East,43.685347,-79.338106,Finesse Cut,43.685971,-79.339749,Barbershop
9,The Danforth East,43.685347,-79.338106,Royal Vacation Travel Tours,43.686176,-79.338472,Travel Agency


El Dataframe contiene 1094 Venues

In [37]:
toronto_venues.shape

(1094, 7)

La cantidad de Venues encontradas en cada barrio varía entre 2 y 50 (el máximo que devuelve FourSquare)

In [38]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton , Parkdale Village , Exhibition Place",19,19,19,19,19,19
Central Bay Street,50,50,50,50,50,50
Christie,10,10,10,10,10,10
Church and Wellesley,50,50,50,50,50,50
"Commerce Court , Victoria Hotel",50,50,50,50,50,50
Davisville,30,30,30,30,30,30
Davisville North,24,24,24,24,24,24
"Dufferin , Dovercourt Village",38,38,38,38,38,38


Hay 240 categorías diferentes

In [39]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


#### Analisis de cada barrio

In [40]:
# codificación
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,ATM,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Art Gallery,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Attorney / Law Office,Audiovisual Service,Auditorium,Australian Restaurant,Automotive Repair Shop,Automotive Retail,Automotive Service,BBQ Joint,Bagel Shop,Bakery,Bank,Banking and Finance,Bar,Barbershop,Bathroom Contractor,Beer Bar,Beer Store,Bicycle Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Store,Bubble Tea Shop,Buffet,Building and Land Surveyor,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafes, Coffee, and Tea Houses",Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Car Dealership,Car Wash and Detail,Caribbean Restaurant,Carpet and Flooring Contractor,Caterer,Cheese Store,Children's Clothing Store,Chinese Restaurant,Chocolate Store,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Book Store,Community and Government,Computer Repair Service,Concert Hall,Construction,Convenience Store,Credit Union,Creperie,Cuban Restaurant,Dance Studio,Deli,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dining and Drinking,Discount Store,Dive Bar,Dog Park,Drugstore,Dry Cleaner,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Fabric / Textiles Store,Falafel Restaurant,Farmers' Market,Fashion Accessories Store,Fast Food Restaurant,Film Studio,Financial Planner,Financial Service,Fish Market,Fish and Chips Shop,Flower Store,Food and Beverage Retail,Food and Beverage Service,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit and Vegetable Store,Furniture and Home Store,Garden,Garden Center,Gay Bar,General Contractor,Gift Store,Golf Course,Gourmet Store,Greek Restaurant,Grocery Store / Supermarket,Hair Salon,Halal Restaurant,Hardware Store,Health Food Store,Health and Beauty Service,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",Hiking Trail,Hobby Store,Home Appliance Store,Home Improvement Service,Home Inspection,Hotel Bar,Ice Cream Parlor,Indian Restaurant,Insurance Agency,Interior Designer,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Landmarks and Outdoors,Landscaper and Gardener,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Legal Service,Liquor Store,Lounge,Mac and Cheese Joint,Management Consultant,Manufacturer,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store,Molecular Gastronomy Restaurant,Monument,Motorcycle Dealership,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Club,Noodle Restaurant,Nutritionist,Optometrist,Organic Grocery,Organization,Other Great Outdoors,Painter,Park,Party Supply Store,Performing Arts Venue,Pet Grooming Service,Pet Supplies Store,Photographer,Photography Lab,Physician,Pizzeria,Playground,Plaza,Poke Restaurant,Polish Restaurant,Pool Hall,Preschool,Print Store,Professional Cleaning Service,Property Management Office,Pub,Public Art,Public and Social Service,Publisher,Ramen Restaurant,Real Estate Agency,Real Estate Service,Record Store,Recording Studio,Restaurant,Retail,Retirement Home,Roof Deck,Salad Restaurant,Seafood Restaurant,"Shipping, Freight, and Material Transportation Service",Shopping Mall,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Retail,Sports Bar,Sports Club,Sports and Recreation,Stadium,Stationery Store,Sushi Restaurant,Swimming Pool,Taco Restaurant,Tailor,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Club,Thai Restaurant,Theater,Tobacco Store,Toy / Game Store,Travel Agency,Used Car Dealership,Vegan and Vegetarian Restaurant,Video Games Store,Vietnamese Restaurant,Vintage and Thrift Store,Wine Bar
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [41]:
toronto_onehot.shape

(1094, 241)

Agrupamos las filas por barrios tomando la média de la frecuancia de la ocurrencia de cada categoría

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Art Gallery,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Attorney / Law Office,Audiovisual Service,Auditorium,Australian Restaurant,Automotive Repair Shop,Automotive Retail,Automotive Service,BBQ Joint,Bagel Shop,Bakery,Bank,Banking and Finance,Bar,Barbershop,Bathroom Contractor,Beer Bar,Beer Store,Bicycle Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Store,Bubble Tea Shop,Buffet,Building and Land Surveyor,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafes, Coffee, and Tea Houses",Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Car Dealership,Car Wash and Detail,Caribbean Restaurant,Carpet and Flooring Contractor,Caterer,Cheese Store,Children's Clothing Store,Chinese Restaurant,Chocolate Store,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Book Store,Community and Government,Computer Repair Service,Concert Hall,Construction,Convenience Store,Credit Union,Creperie,Cuban Restaurant,Dance Studio,Deli,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dining and Drinking,Discount Store,Dive Bar,Dog Park,Drugstore,Dry Cleaner,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Fabric / Textiles Store,Falafel Restaurant,Farmers' Market,Fashion Accessories Store,Fast Food Restaurant,Film Studio,Financial Planner,Financial Service,Fish Market,Fish and Chips Shop,Flower Store,Food and Beverage Retail,Food and Beverage Service,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit and Vegetable Store,Furniture and Home Store,Garden,Garden Center,Gay Bar,General Contractor,Gift Store,Golf Course,Gourmet Store,Greek Restaurant,Grocery Store / Supermarket,Hair Salon,Halal Restaurant,Hardware Store,Health Food Store,Health and Beauty Service,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",Hiking Trail,Hobby Store,Home Appliance Store,Home Improvement Service,Home Inspection,Hotel Bar,Ice Cream Parlor,Indian Restaurant,Insurance Agency,Interior Designer,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Landmarks and Outdoors,Landscaper and Gardener,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Legal Service,Liquor Store,Lounge,Mac and Cheese Joint,Management Consultant,Manufacturer,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store,Molecular Gastronomy Restaurant,Monument,Motorcycle Dealership,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Club,Noodle Restaurant,Nutritionist,Optometrist,Organic Grocery,Organization,Other Great Outdoors,Painter,Park,Party Supply Store,Performing Arts Venue,Pet Grooming Service,Pet Supplies Store,Photographer,Photography Lab,Physician,Pizzeria,Playground,Plaza,Poke Restaurant,Polish Restaurant,Pool Hall,Preschool,Print Store,Professional Cleaning Service,Property Management Office,Pub,Public Art,Public and Social Service,Publisher,Ramen Restaurant,Real Estate Agency,Real Estate Service,Record Store,Recording Studio,Restaurant,Retail,Retirement Home,Roof Deck,Salad Restaurant,Seafood Restaurant,"Shipping, Freight, and Material Transportation Service",Shopping Mall,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Retail,Sports Bar,Sports Club,Sports and Recreation,Stadium,Stationery Store,Sushi Restaurant,Swimming Pool,Taco Restaurant,Tailor,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Club,Thai Restaurant,Theater,Tobacco Store,Toy / Game Store,Travel Agency,Used Car Dealership,Vegan and Vegetarian Restaurant,Video Games Store,Vietnamese Restaurant,Vintage and Thrift Store,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.02,0.020000,0.0

In [43]:
toronto_grouped.shape

(38, 241)

Imprimimos cada barrio junto con los 5 sitios mas comunes

In [44]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.06
1                  Pub  0.06
2             Beer Bar  0.04
3  Fish and Chips Shop  0.04
4                 Café  0.04


----Brockton , Parkdale Village , Exhibition Place----
                     venue  freq
0                     Café  0.11
1       Pet Supplies Store  0.05
2  Landscaper and Gardener  0.05
3                     Park  0.05
4          Photography Lab  0.05


----Central Bay Street----
                           venue  freq
0  Cafes, Coffee, and Tea Houses  0.12
1                           Deli  0.08
2                    Coffee Shop  0.08
3           Fast Food Restaurant  0.04
4                           Café  0.04


----Christie----
                         venue  freq
0                         Café   0.2
1  Grocery Store / Supermarket   0.2
2                       Retail   0.1
3                Bicycle Store   0.1
4             Toy / Game Store   0.1


----Church and Wellesley----
                

Escribimos una función para ordenar los sitios en orden descendente.

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Generamos el nuevo dataframe y mostramos los primeros 10 sitios de cada barrio.

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Pub,Beer Bar,Fish and Chips Shop,Café,Drugstore,Night Club,Farmers' Market,Liquor Store,Bistro
1,"Brockton , Parkdale Village , Exhibition Place",Café,Pet Supplies Store,Landscaper and Gardener,Park,Photography Lab,Playground,Deli,Dining and Drinking,Electronics Store,Retirement Home
2,Central Bay Street,"Cafes, Coffee, and Tea Houses",Deli,Coffee Shop,Fast Food Restaurant,Café,Diner,Japanese Restaurant,Pizzeria,Thai Restaurant,Juice Bar
3,Christie,Café,Grocery Store / Supermarket,Retail,Bicycle Store,Toy / Game Store,Candy Store,Laundry Service,Landmarks and Outdoors,ATM,Monument
4,Church and Wellesley,Diner,Hair Salon,Pizzeria,Italian Restaurant,Café,Restaurant,Clothing Store,Bubble Tea Shop,Liquor Store,Thai Restaurant
5,"Commerce Court , Victoria Hotel",Restaurant,Café,"Cafes, Coffee, and Tea Houses",Bakery,Coffee Shop,Diner,Pub,Bistro,Burger Joint,Bank
6,Davisville,Health and Beauty Service,Flower Store,Toy / Game Store,Italian Restaurant,Bank,Coffee Shop,Retail,Candy Store,Seafood Restaurant,Spa
7,Davisville North,Attorney / Law Office,Landscaper and Gardener,Grocery Store / Supermarket,Convenience Store,Golf Course,Business and Professional Services,Restaurant,Building and Land Surveyor,Real Estate Agency,Flower Store
8,"Dufferin , Dovercourt Village",Photographer,Sports and Recreation,Hair Salon,Liquor Store,Health Food Store,Park,Soccer Field,Shopping Mall,Recording Studio,Car Wash and Detail
9,Enclave of M4L,Brewery,Health and Beauty Service,Automotive Service,Farmers' Market,Pizzeria,Coffee Shop,Credit Union,"Cafes, Coffee, and Tea Houses",Motorcycle Dealership,Museum


#### Agrupar los barrios

In [47]:
# establecer el número de agrupaciones
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_ 

C:\Users\u199401\AppData\Local\Temp/ipykernel_16284/2986415486.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 2,
       1, 3, 3, 0, 3, 3, 3, 3, 3, 3, 4, 0, 3, 3, 3, 3])

Generamos un nuevo dataframe que incluya la agrupación asi como los 10 sitios mas populares de cada barrio.

In [48]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto_pc

# juntar manhattan_grouped con manhattan_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Hiking Trail,Park,Grocery Store / Supermarket,Sports and Recreation,Film Studio,Audiovisual Service,ATM,Miscellaneous Store,Motorcycle Dealership,Monument
1,40,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,3.0,Playground,Travel Agency,Jewelry Store,Barbershop,Bathroom Contractor,Landscaper and Gardener,Toy / Game Store,Sports and Recreation,Event Service,Park
2,41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,3.0,Restaurant,Greek Restaurant,Ice Cream Parlor,Bakery,Bookstore,Hair Salon,Café,Retail,Italian Restaurant,Diner
3,42,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,3.0,Pizzeria,Pet Grooming Service,Burger Joint,Automotive Repair Shop,Business and Professional Services,Liquor Store,Ice Cream Parlor,Park,Convenience Store,Music Venue
4,43,M4M,East Toronto,Studio District,43.659526,-79.340923,3.0,Bar,Coffee Shop,Retail,Hair Salon,Bakery,Drugstore,Nail Salon,Italian Restaurant,Restaurant,Ice Cream Parlor
5,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3.0,Business and Strategy Consulting Office,Architecture Firm,Construction,Interior Designer,Bank,Bakery,Dim Sum Restaurant,Legal Service,Tennis Club,Spa
6,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3.0,Attorney / Law Office,Landscaper and Gardener,Grocery Store / Supermarket,Convenience Store,Golf Course,Business and Professional Services,Restaurant,Building and Land Surveyor,Real Estate Agency,Flower Store
7,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2.0,Sushi Restaurant,Audiovisual Service,ATM,Music Venue,Manufacturer,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store
8,47,M4S,Central Toronto,Davisville,43.704324,-79.388790,3.0,Health and Beauty Service,Flower Store,Toy / Game Store,Italian Restaurant,Bank,Coffee Shop,Retail,Candy Store,Seafood Restaurant,Spa
9,48,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160,0.0,Park,Home Inspection,Financial Service,Business and Professional Services,ATM,Music Store,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant


Visualizamos las agrupaciones resultantes

In [49]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Agrupacion 0: Parques y paseos

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,-79.293031,0.0,Hiking Trail,Park,Grocery Store / Supermarket,Sports and Recreation,Film Studio,Audiovisual Service,ATM,Miscellaneous Store,Motorcycle Dealership,Monument
9,M4T,-79.383160,0.0,Park,Home Inspection,Financial Service,Business and Professional Services,ATM,Music Store,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
11,M4W,-79.377529,0.0,Park,Swimming Pool,ATM,Museum,Management Consultant,Manufacturer,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
35,M6P,-79.464763,0.0,Park,Art Gallery,Bar,Business and Professional Services,ATM,Molecular Gastronomy Restaurant,Museum,Movie Theater,Motorcycle Dealership,Monument


#### Agrupacion 1: Un barrio particular en las afueras, con negocios de servicios audiovisuales.

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,M6R,-79.456325,1.0,Advertising Agency,Film Studio,Garden,Music Store,Management Consultant,Manufacturer,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant


#### Agrupacion 2: Un barrio particular en las afueras, con restaurants de sushi.

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M4R,-79.405678,2.0,Sushi Restaurant,Audiovisual Service,ATM,Music Venue,Manufacturer,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store


#### Agrupacion 3: Bares, cafes y entretenimiento

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4J,-79.338106,3.0,Playground,Travel Agency,Jewelry Store,Barbershop,Bathroom Contractor,Landscaper and Gardener,Toy / Game Store,Sports and Recreation,Event Service,Park
2,M4K,-79.352188,3.0,Restaurant,Greek Restaurant,Ice Cream Parlor,Bakery,Bookstore,Hair Salon,Café,Retail,Italian Restaurant,Diner
3,M4L,-79.315572,3.0,Pizzeria,Pet Grooming Service,Burger Joint,Automotive Repair Shop,Business and Professional Services,Liquor Store,Ice Cream Parlor,Park,Convenience Store,Music Venue
4,M4M,-79.340923,3.0,Bar,Coffee Shop,Retail,Hair Salon,Bakery,Drugstore,Nail Salon,Italian Restaurant,Restaurant,Ice Cream Parlor
5,M4N,-79.388790,3.0,Business and Strategy Consulting Office,Architecture Firm,Construction,Interior Designer,Bank,Bakery,Dim Sum Restaurant,Legal Service,Tennis Club,Spa
6,M4P,-79.390197,3.0,Attorney / Law Office,Landscaper and Gardener,Grocery Store / Supermarket,Convenience Store,Golf Course,Business and Professional Services,Restaurant,Building and Land Surveyor,Real Estate Agency,Flower Store
8,M4S,-79.388790,3.0,Health and Beauty Service,Flower Store,Toy / Game Store,Italian Restaurant,Bank,Coffee Shop,Retail,Candy Store,Seafood Restaurant,Spa
10,M4V,-79.400049,3.0,Financial Planner,Fabric / Textiles Store,Sports and Recreation,Advertising Agency,Dry Cleaner,Cheese Store,Public and Social Service,Drugstore,Bagel Shop,Real Estate Agency
12,M4X,-79.367675,3.0,Convenience Store,Chinese Restaurant,Retail,Massage Clinic,Butcher,"Cafes, Coffee, and Tea Houses",Ethiopian Restaurant,Dry Cleaner,Bakery,Jewelry Store
13,M4Y,-79.383160,3.0,Diner,Hair Salon,Pizzeria,Italian Restaurant,Café,Restaurant,Clothing Store,Bubble Tea Shop,Liquor Store,Thai Restaurant


#### Agrupacion 4: Un barrio particular en el centro, con restaurants de comida india.

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M5R,-79.405678,4.0,South Indian Restaurant,Café,"Shipping, Freight, and Material Transportation...",ATM,Museum,Management Consultant,Manufacturer,Massage Clinic,Mediterranean Restaurant,Mexican Restaurant
